In [120]:
import os
import sys

from pyspark import SparkContext, SparkConf, Row, StorageLevel

SPARK_HOME = '/home/hadoop'  + '/spark'

os.environ['SPARK_HOME'] = SPARK_HOME
sys.path.append(SPARK_HOME + "/python")
sys.path.append(SPARK_HOME + "/python/lib/py4j-0.10.7-src.zip")

master = "spark://nodemaster:7077" 

In [2]:
try:
    sc.stop()
except Exception as e:
    print('no sc')
    
app_name = "spark_demo"

conf = SparkConf().setAppName(app_name).setMaster(master)
try:
    sc = SparkContext.getOrCreate()
except:
    sc = SparkContext(conf=conf)
#     pass

no sc


In [3]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [4]:
# -*- coding: utf-8 -*-

def getTest(sc):
    data = [1, 2, 3, 4, 5]
    distData = sc.parallelize(data)
    res = distData.reduce(lambda a, b: a + b)
#     print (res)
    return res
def textfile():
    text_file = sc.textFile("hdfs:///sparkdir/dfile1")
    print(text_file.collect())
    df = text_file.map(lambda r: Row(r)).toDF(["line"])
    print(df)

In [5]:
getTest(sc)

15

In [8]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [9]:
%ls -al /home/hadoop/devlop/spark.py

-rw-r--r-- 1 root root 949 Oct 15 05:52 /home/hadoop/devlop/spark.py


从本地读取文件

In [10]:
textFile=sc.textFile("file:/home/hadoop/devlop/test.txt")

In [11]:
textFile

file:/home/hadoop/devlop/test.txt MapPartitionsRDD[3] at textFile at NativeMethodAccessorImpl.java:0

In [14]:
textFile.count()

1

基本RDD转换

In [15]:
intRDD = sc.parallelize([3,1, 2, 5, 5])
intRDD.collect()

[3, 1, 2, 5, 5]

In [16]:
stringRDD = sc.parallelize(["Apple", "Orange", "Banana","Grape","Apple"])
stringRDD.collect()

['Apple', 'Orange', 'Banana', 'Grape', 'Apple']

In [17]:
def addOne(x):
    return (x+1)

intRDD.map(addOne).collect()

[4, 2, 3, 6, 6]

In [18]:
intRDD.map(lambda  x : x + 1).collect()

[4, 2, 3, 6, 6]

In [21]:
stringRDD.map(lambda x: "fruit:"+x).collect()

['fruit:Apple', 'fruit:Orange', 'fruit:Banana', 'fruit:Grape', 'fruit:Apple']

In [22]:
stringRDD.filter(lambda x : "ra" in  x ).collect()

['Orange', 'Grape']

In [23]:
intRDD.filter(lambda x : x < 3).collect()

[1, 2]

In [24]:
intRDD.filter(lambda x : x == 3).collect()

[3]

In [25]:
intRDD.filter(lambda x : 1< x and x <5  ).collect()

[3, 2]

In [26]:
intRDD.filter(lambda x : x >=5 or x<3 ).collect()

[1, 2, 5, 5]

In [27]:
stringRDD.filter(lambda x : "ra" in x ).collect()

['Orange', 'Grape']

In [28]:
intRDD.distinct().collect()

[1, 5, 2, 3]

In [32]:
type(intRDD.collect())
intRDD.collect()

[3, 1, 2, 5, 5]

In [33]:
stringRDD.distinct().collect()

['Orange', 'Apple', 'Grape', 'Banana']

In [34]:
sRDD = intRDD.randomSplit([0.4,0.6])

In [35]:
sRDD[0].collect()

[3, 5]

In [36]:
sRDD

[PythonRDD[29] at collect at <ipython-input-35-2f1eef4eaf0f>:1,
 PythonRDD[30] at RDD at PythonRDD.scala:52]

In [41]:
sRDD[1].collect()

[1, 2, 5]

In [42]:
gRDD=intRDD.groupBy(
    lambda  x: "even" if (x % 2 == 0)     else "odd"
).collect()

In [43]:
len(gRDD)

2

In [55]:
gRDD[0][0], list(gRDD[0][1])
gRDD[1][0], list(gRDD[1][1])

('odd', [3, 1, 5, 5])

# 10.4 多个RDD“转换”运算

In [56]:
intRDD1 = sc.parallelize([3, 1, 2, 5, 5])
intRDD2 = sc.parallelize([5, 6])
intRDD3 = sc.parallelize([2, 7])

In [57]:
intRDD1.collect()

[3, 1, 2, 5, 5]

In [58]:
intRDD2.collect()

[5, 6]

In [59]:
intRDD3.collect()

[2, 7]

In [60]:
intRDD1.union(intRDD2).union(intRDD3).collect()

[3, 1, 2, 5, 5, 5, 6, 2, 7]

In [61]:
intRDD1.intersection(intRDD2).collect()

[5]

In [62]:
intRDD1.subtract(intRDD2).collect()

[1, 2, 3]

In [64]:
print(intRDD1.cartesian(intRDD2).collect())

[(3, 5), (3, 6), (1, 5), (1, 6), (2, 5), (2, 6), (5, 5), (5, 5), (5, 6), (5, 6)]


# 10.5 基本“动作”运算

In [65]:
#Step1 读取元素

In [68]:
intRDD.collect()

[3, 1, 2, 5, 5]

In [69]:
intRDD.first()

3

In [70]:
intRDD.take(2)

[3, 1]

In [71]:
intRDD.takeOrdered(3)

[1, 2, 3]

In [72]:
intRDD.takeOrdered(3,key=lambda x: -x)

[5, 5, 3]

In [73]:
intRDD.stats()

(count: 5, mean: 3.2, stdev: 1.6, max: 5, min: 1)

In [74]:
intRDD.min()

1

In [75]:
intRDD.max()

5

In [76]:
intRDD.stdev()

1.6

In [77]:
intRDD.count()

5

In [78]:
intRDD.sum()

16

In [79]:
intRDD.mean()

3.2

# 10.6 RDD Key-Value 基本“转换”运算

In [80]:
kvRDD1 = sc.parallelize([(3, 4), (3, 6), (5, 6), (1, 2)])
kvRDD1.collect()

[(3, 4), (3, 6), (5, 6), (1, 2)]

In [81]:
kvRDD1.keys().collect()

[3, 3, 5, 1]

In [82]:
kvRDD1.values().collect()

[4, 6, 6, 2]

In [83]:
kvRDD1.filter(lambda keyValue: keyValue[0] < 5).collect()

[(3, 4), (3, 6), (1, 2)]

In [84]:
kvRDD1.filter(lambda keyValue: keyValue[1] < 5).collect()

[(3, 4), (1, 2)]

In [85]:
kvRDD1.mapValues(lambda x : x * x).collect()

[(3, 16), (3, 36), (5, 36), (1, 4)]

In [86]:
kvRDD1.sortByKey(ascending=True).collect()

[(1, 2), (3, 4), (3, 6), (5, 6)]

In [87]:
kvRDD1.sortByKey().collect()

[(1, 2), (3, 4), (3, 6), (5, 6)]

In [88]:
kvRDD1.sortByKey(ascending=False).collect()

[(5, 6), (3, 4), (3, 6), (1, 2)]

In [89]:
kvRDD1.reduceByKey(lambda x ,y: x+y).collect()

[(1, 2), (5, 6), (3, 10)]

## 10.7 多个RDD Key-Value“转换”运算

In [90]:
kvRDD1 = sc.parallelize([(3, 4), (3, 6), (5, 6), (1, 2)])
kvRDD2 = sc.parallelize([(3, 8)])

In [91]:
kvRDD1.collect()

[(3, 4), (3, 6), (5, 6), (1, 2)]

In [92]:
kvRDD2.collect()

[(3, 8)]

In [93]:
kvRDD1.join(kvRDD2).collect()

[(3, (4, 8)), (3, (6, 8))]

In [94]:
kvRDD1.leftOuterJoin(kvRDD2).collect()

[(1, (2, None)), (3, (4, 8)), (3, (6, 8)), (5, (6, None))]

In [95]:
kvRDD1.rightOuterJoin(kvRDD2).collect()

[(3, (4, 8)), (3, (6, 8))]

In [96]:
kvRDD1.subtractByKey(kvRDD2).collect()

[(1, 2), (5, 6)]

## 10.8 Key-Value “动作”运算

In [97]:
kvRDD1.first()

(3, 4)

In [98]:
kvRDD1.take(2)

[(3, 4), (3, 6)]

In [99]:
kvFirst=kvRDD1.first()
kvFirst

(3, 4)

In [100]:
kvFirst[0]

3

In [101]:
kvFirst[1]

4

In [102]:
kvRDD1.countByKey()

defaultdict(int, {1: 1, 3: 2, 5: 1})

In [103]:
KV=kvRDD1.collectAsMap()
KV

{1: 2, 3: 6, 5: 6}

In [104]:
type(KV)

dict

In [105]:
KV[3]

6

In [106]:
KV[1]

2

In [107]:
kvRDD1.lookup(3)

[4, 6]

In [108]:
kvRDD1.lookup(5)

[6]

# 10.9 Broadcast 广播变量

In [109]:
kvFruit = sc.parallelize([(1, "apple"), (2, "orange"), (3, "banana"), (4, "grape")])

In [111]:
fruitMap=kvFruit.collectAsMap()
print("字典："+str(fruitMap))

字典：{1: 'apple', 2: 'orange', 3: 'banana', 4: 'grape'}


In [177]:
fruitIds=sc.parallelize([2,4,1,3])
print "水果编号："+str(fruitIds.collect()),

水果编号：[2, 4, 1, 3]


In [178]:
print "使用字典进行转换=>"
fruitNames= fruitIds.map(lambda x : fruitMap[x]).collect()
print "水果名称："+str(fruitNames)

使用字典进行转换=>
水果名称：['orange', 'grape', 'apple', 'banana']


In [179]:
kvFruit = sc.parallelize([(1, "apple"), (2, "orange"), (3, "banana"), (4, "grape")])

In [180]:
fruitMap=kvFruit.collectAsMap()
bcFruitMap=sc.broadcast(fruitMap)
print "字典："+str(fruitMap)

字典：{1: 'apple', 2: 'orange', 3: 'banana', 4: 'grape'}


In [181]:
fruitIds=sc.parallelize([2,4,1,3])
print "水果编号："+str(fruitIds.collect()),

水果编号：[2, 4, 1, 3]


In [182]:
print "使用Broadcast 广播变量字典进行转换=>"
fruitNames= fruitIds.map(lambda x : bcFruitMap.value[x] ).collect()
print "水果名称："+str(fruitNames)

使用Broadcast 广播变量字典进行转换=>
水果名称：['orange', 'grape', 'apple', 'banana']


# 10.10	accumulator累加器

In [183]:
intRDD = sc.parallelize([3,1, 2, 5, 5])

In [184]:
total = sc.accumulator(0.0)

In [185]:
num = sc.accumulator(0)

In [186]:
intRDD.foreach(lambda  i:  [total.add(i) , num.add(1)])

In [187]:
avg = total.value / num.value        
print("total="+str(total.value)+", num="+str(num.value)+", avg="+str(avg))

total=16.0, num=5, avg=3.2


In [188]:
intRDD = sc.parallelize([3,1, 2, 5, 5])

total = sc.accumulator(0.0)
num = sc.accumulator(0)

intRDD.foreach(lambda  i:  [total.add(i) , num.add(1)])

avg = total.value / num.value        
print("total="+str(total.value)+", num="+str(num.value)+", avg="+str(avg))

total=16.0, num=5, avg=3.2


# 10.11 RDD Persistence持久化

In [112]:
intRddMemory = sc.parallelize([3,1, 2, 5, 5])

In [113]:
intRddMemory.persist()

ParallelCollectionRDD[145] at parallelize at PythonRDD.scala:194

In [114]:
intRddMemory.is_cached

True

In [115]:
intRddMemory.unpersist()

ParallelCollectionRDD[145] at parallelize at PythonRDD.scala:194

In [116]:
intRddMemory.is_cached

False

In [117]:
intRddMemoryAndDisk = sc.parallelize([3,1, 2, 5, 5])

In [121]:
intRddMemoryAndDisk.persist(StorageLevel.MEMORY_AND_DISK)

ParallelCollectionRDD[146] at parallelize at PythonRDD.scala:194

In [122]:
intRddMemoryAndDisk.is_cached

True

# 10.12 使用Spark 建立WordCount 

In [124]:
%rm -R /home/hadoop/devlop/architecture/learn_docker/pythonspark/ipynotebook/data/output

rm: cannot remove '/home/hadoop/devlop/architecture/learn_docker/pythonspark/ipynotebook/data/output': No such file or directory


In [125]:
%pwd

'/home/hadoop/devlop'

In [126]:
textFile = sc.textFile("architecture/learn_docker/pythonspark/ipynotebook/data/test.txt")
stringRDD=textFile.flatMap(lambda line : line.split(" "))
countsRDD = stringRDD.map(lambda word : (word, 1))  \
                                            .reduceByKey(lambda x,y : x+y)
countsRDD.saveAsTextFile("data/output")

Py4JJavaError: An error occurred while calling o935.partitions.
: java.net.ConnectException: Call From nodemaster/202.204.60.242 to nodemaster:9000 failed on connection exception: java.net.ConnectException: Connection refused; For more details see:  http://wiki.apache.org/hadoop/ConnectionRefused
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:423)
	at org.apache.hadoop.net.NetUtils.wrapWithMessage(NetUtils.java:831)
	at org.apache.hadoop.net.NetUtils.wrapException(NetUtils.java:755)
	at org.apache.hadoop.ipc.Client.getRpcResponse(Client.java:1501)
	at org.apache.hadoop.ipc.Client.call(Client.java:1443)
	at org.apache.hadoop.ipc.Client.call(Client.java:1353)
	at org.apache.hadoop.ipc.ProtobufRpcEngin

In [200]:
%ll data/output

总计 4
-rw-r--r-- 1 hduser 58  9月 26 18:43 part-00000
-rw-r--r-- 1 hduser  0  9月 26 18:43 _SUCCESS


In [201]:
%cat data/output/part-00000

(u'Orange', 1)
(u'Grape', 2)
(u'Apple', 2)
(u'Banana', 1)


In [202]:
%cat data/output/part-00001

cat: data/output/part-00001: 没有这个文件或目录


In [203]:
%rm -R data/output

# 10.13 Spark WordCount 详细解说 

In [204]:
#Step1 sc.textFile读取文件 

In [205]:
textFile = sc.textFile("data/test.txt")
textFile.collect()

[u'Apple Apple Orange', u'Banana Grape Grape']

In [206]:
#Step2 flatMap取出每一个单词

In [207]:
stringRDD=textFile.map(lambda line : line.split(" "))
stringRDD.collect()

[[u'Apple', u'Apple', u'Orange'], [u'Banana', u'Grape', u'Grape']]

In [208]:
stringRDD=textFile.flatMap(lambda line : line.split(" "))
stringRDD.collect()

[u'Apple', u'Apple', u'Orange', u'Banana', u'Grape', u'Grape']

In [209]:
stringRDD.map(lambda word : (word, 1)).collect()

[(u'Apple', 1),
 (u'Apple', 1),
 (u'Orange', 1),
 (u'Banana', 1),
 (u'Grape', 1),
 (u'Grape', 1)]

In [210]:
stringRDD.map(lambda word : (word, 1)).reduceByKey(lambda x,y : x+y).collect()

[(u'Orange', 1), (u'Grape', 2), (u'Apple', 2), (u'Banana', 1)]

In [127]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [128]:
sc

<SparkContext master=local[*] appName=pyspark-shell>

In [130]:
sc.master

'local[*]'